<a href="https://colab.research.google.com/github/cerr/CERR/blob/octave_dev/Jupyter_Notebooks/demo_DLseg_swallowing_and_chewing_structures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Deep learning-based segmentation of Swallowing and Chewing structures in H&N CT scans

This notebook demonstrates the application of a Convolution Neural Network model to segment the following swallowing and chewing structures in H&N CT scans. Model details are available in [bioRxiv article](https://www.biorxiv.org/content/10.1101/772178v1) by Iyer A *et al*.

### Output structures:

*   Masseters (left, right)
*   Medial pterygoid (left, right)
*   Larynx_DLabV3
*   Constrictor muscle

---


## Install Octave 5.2 and required packages

In [ ]:
%%capture
! apt-get update
! apt-get install gcc g++ gfortran make libblas-dev liblapack-dev libpcre3-dev libarpack2-dev libcurl4-gnutls-dev epstool libfftw3-dev transfig libfltk1.3-dev libfontconfig1-dev libfreetype6-dev libgl2ps-dev libglpk-dev libreadline-dev gnuplot-x11 libgraphicsmagick++1-dev libhdf5-serial-dev openjdk-8-jdk libsndfile1-dev llvm-dev lpr texinfo libgl1-mesa-dev libosmesa6-dev pstoedit portaudio19-dev libqhull-dev libqrupdate-dev libqscintilla2-dev libsuitesparse-dev texlive texlive-generic-recommended libxft-dev zlib1g-dev autoconf automake bison flex gperf gzip icoutils librsvg2-bin libtool perl rsync tar qtbase5-dev qttools5-dev qttools5-dev-tools libqscintilla2-qt5-dev wget git libsundials-dev gnuplot x11-apps

In [ ]:
%%capture
%%bash
LOCATION=$(curl -s https://api.github.com/repos/cerr/octave-colab/tags \
| grep "name" | tail -n 1 \
| awk '{print "https://github.com/cerr/octave-colab/archive/" substr($2, 2, length($2)-3) ".zip"}') \
; curl -L -o octavecolab.zip $LOCATION

unzip octavecolab.zip -d octavecolab
releaseDir=$(ls octavecolab)
tar xzvf "octavecolab/${releaseDir}/octavecolab.tar.gz"

In [ ]:
import os
os.environ['OCTAVE_EXECUTABLE'] = '/content/octavecolab/bin/octave-cli'
os.environ['PATH'] = '/content/octavecolab/bin:' + os.environ['PATH']

In [ ]:
%%capture
! pip3 install octave_kernel
! pip3 install oct2py

%load_ext oct2py.ipython

!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/image-2.12.0.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/io-2.6.1.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/statistics-1.4.2.tar.gz

!octave --eval "pkg install image-2.12.0.tar.gz"
!octave --eval "pkg install io-2.6.1.tar.gz"
!octave --eval "pkg install statistics-1.4.2.tar.gz"

## Download pretrained models

In [ ]:
%%capture
%%bash
LOCATION=$(curl -s https://api.github.com/repos/aditiiyer/CT_SwallowingAndChewing_DeepLabV3/tags \
| grep "name"| grep "linux" \
| awk '{print "https://github.com/aditiiyer/CT_SwallowingAndChewing_DeepLabV3/archive/" substr($2, 2, length($2)-3) ".zip"}') \
; curl -L -o swallowChewSeg.zip $LOCATION

unzip swallowChewSeg.zip -d pretrained


## Download CERRx

In [ ]:
%%capture
!git clone --single-branch --branch octave_dev https://www.github.com/cerr/CERR.git

## Apply segmentation models to sample data

In [ ]:
from oct2py import octave

octave.eval('pkg load image')
octave.eval('pkg load io')
octave.eval('pkg load statistics')
octave.addpath(octave.genpath('/content/CERR'))

In [ ]:
sampleData = '/content/CERR/Unit_Testing/data_for_cerr_tests/' + \
              'CERR_plans/head_neck_ex1_20may03.mat.bz2'

%octave_push sampleData
planC = octave.loadPlanC(sampleData, octave.tempdir())
planC = octave.updatePlanFields(planC)
planC = octave.quality_assure_planC(sampleData, planC)

algorithm = 'CT_ChewingStructures_DeepLabV3^CT_Larynx_DeepLabV3^' + \
            'CT_PharyngealConstrictor_DeepLabV3';
sessionPath = '/content/temp/'
condaEnvName = '/content/pretrained/CT_SwallowingAndChewing_DeepLabV3-1.0-linux/'
planC = octave.runSegForPlanCInCondaEnv(planC,sessionPath,algorithm, \
                                        condaEnvName)

In [ ]:
from oct2py import octave
%octave_push planC

In [ ]:
%%octave

addpath(genpath('/content/CERR'))

%Get scan array
indexS = planC{end};
scanNum = 1;
ctOffset = planC{indexS.scan}(scanNum).scanInfo(1).CTOffset;
scanArray = single(getScanArray(scanNum,planC)) - ctOffset;

%Get structure labels & masks
numStructs = length(planC{indexS.structures});
structNameC = {planC{indexS.structures}.structureName};
strNameC = {'Left_masseter', 'Right_masseter', 'Left_medial_pterygoid',...
              'Right_medial_pterygoid', 'Larynx_DLabV3', 'Constrictor_muscle'};
for strNum = 1:length(strNameC)
    strx = strNameC{strNum};
    idx = getMatchingIndex(strx,structNameC,'EXACT');
    mask3M = getStrMask(idx, planC);
    maskC{strNum} = mask3M;
end

In [ ]:
%octave_pull maskC strNameC scanArray

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import ipywidgets as widgets

dx, dy = 1, 1

x = np.arange(0, 255, dx)
y = np.arange(0, 255, dy)
extent = np.min(x), np.max(x), np.min(y), np.max(y)

clear_output(wait=True)

def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    
    return window_image

def show_axial_slice(slcNum):
    clear_output(wait=True)
    print('Slice '+str(slcNum))
    if 'fig' in locals():
        fig.remove()
    fig, ax = plt.subplots()
    window_center = 45
    window_width = 125
    windowed_img = window_image(scanArray[:,:,slcNum-1],
                                window_center,window_width)
    im1 = ax.imshow(windowed_img, cmap=plt.cm.gray, alpha=1,
                    interpolation='nearest', extent=extent)
    
    cmaps = [plt.cm.Oranges,plt.cm.Oranges,plt.cm.Blues,plt.cm.Blues, \
             plt.cm.Purples,plt.cm.Greens]

    for maskNum in range(0,6,1):
        mask_cmap = cmaps[maskNum]
        mask_cmap.set_under('k', alpha=0)
        im2 = ax.imshow(maskC[0,maskNum][:,:,slcNum-1], 
                        cmap=mask_cmap, alpha=.8, extent=extent,
                        interpolation='none', clim=[0.5, 1])        
    plt.show()

slice_slider = widgets.IntSlider(min=1,max=79,step=1)
outputSlc = widgets.Output()

display(slice_slider, outputSlc)

def update_slice(change):
    with outputSlc:
        show_axial_slice(change['new'])

slice_slider.observe(update_slice, names='value')
